In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [11]:
# 1. 学習データ（スペース区切りの日本語テキスト）
# テキストデータ（X）
train_data = [
    "ここ の ランチ は 最高 で 美味しい",  # ポジティブ
    "店員 の 態度 が 悪い 最悪",         # ネガティブ
    "とても 美味しい 素晴らしい 食事",    # ポジティブ
    "味 が 薄い まずい 遅い"             # ネガティブ
]

In [12]:
# 正解ラベル（y）
train_labels = ["positive", "negative", "positive", "negative"]

In [13]:
# 2. データをベクトル化（文字を数字の行列に変換）
# CountVectorizer: 単語の出現回数を数えて行列にするツール
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data)

In [14]:
# 3. モデルの学習（ナイーブベイズ分類器）
# MultinomialNB: テキスト分類によく使われる多項ナイーブベイズ
model = MultinomialNB()
model.fit(X_train, train_labels)

MultinomialNB()

In [15]:
# 4. 予測してみる
test_text = ["ランチ は 美味しい が 店員 が 遅い"] # ポジティブとネガティブが混ざった文
X_test = vectorizer.transform(test_text)

In [16]:
# 予測実行
prediction = model.predict(X_test)
probability = model.predict_proba(X_test)

In [17]:
print(f"テスト文: {test_text[0]}")
print(f"判定結果: {prediction[0]}")
print(f"確率詳細: Negative={probability[0][0]:.2f}, Positive={probability[0][1]:.2f}")

テスト文: ランチ は 美味しい が 店員 が 遅い
判定結果: positive
確率詳細: Negative=0.45, Positive=0.55


In [18]:
import pandas as pd
import numpy as np

# 1. 各クラスの名前を確認（['negative', 'positive'] の順に入っています）
classes = model.classes_
print(f"クラスの順序: {classes}")

# 2. 単語リストを取得
feature_names = vectorizer.get_feature_names_out()

# 3. 学習したスコア（対数確率）を取得
# model.feature_log_prob_[0] が Negative、[1] が Positive のスコアです
# そのままだと対数（-3.5など）で分かりにくいので、expで元の確率（%）に戻します
probs = np.exp(model.feature_log_prob_)

# 4. 表（DataFrame）にまとめる
df_result = pd.DataFrame(data=probs.T, index=feature_names, columns=classes)

# 見やすくするために「ポジティブ度（倍率）」を追加
# (Positiveの確率 / Negativeの確率) で、どちらに偏っているか算出
df_result['positive_ratio'] = df_result['positive'] / df_result['negative']

# 結果の表示
print("--- 学習された単語ごとの確率（トップ5） ---")
print(df_result.round(4)) # 小数点4桁で丸めて表示

クラスの順序: ['negative' 'positive']
--- 学習された単語ごとの確率（トップ5） ---
       negative  positive  positive_ratio
ここ       0.0476    0.0909          1.9091
とても      0.0476    0.0909          1.9091
まずい      0.0952    0.0455          0.4773
ランチ      0.0476    0.0909          1.9091
店員       0.0952    0.0455          0.4773
悪い       0.0952    0.0455          0.4773
態度       0.0952    0.0455          0.4773
最悪       0.0952    0.0455          0.4773
最高       0.0476    0.0909          1.9091
素晴らしい    0.0476    0.0909          1.9091
美味しい     0.0476    0.1364          2.8636
薄い       0.0952    0.0455          0.4773
遅い       0.0952    0.0455          0.4773
食事       0.0476    0.0909          1.9091
